In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from imgaug import augmenters as iaa
from tqdm import tqdm_notebook as tqdm
import json
import copy
import mmcv
from multiprocessing import Pool
import sys
import skimage.io as io
from pycocotools.coco import COCO   # 载入 cocoz
%matplotlib inline
import random
from PIL import Image
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [16]:
# def bbox_overlaps_py(boxes, query_boxes):
#     """
#     determine overlaps between boxes and query_boxes
#     :param boxes: n * 4 bounding boxes
#     :param query_boxes: k * 4 bounding boxes
#     :return: overlaps: n * k overlaps
#     """
#     n_ = boxes.shape[0]
#     k_ = query_boxes.shape[0]
#     overlaps = np.zeros((n_, k_), dtype=np.float)
#     for k in range(k_):
#         query_box_area = (query_boxes[k, 2] - query_boxes[k, 0] + 1) * (query_boxes[k, 3] - query_boxes[k, 1] + 1)
#         for n in range(n_):
#             iw = min(boxes[n, 2], query_boxes[k, 2]) - max(boxes[n, 0], query_boxes[k, 0]) + 1
#             if iw > 0:
#                 ih = min(boxes[n, 3], query_boxes[k, 3]) - max(boxes[n, 1], query_boxes[k, 1]) + 1
#                 if ih > 0:
#                     box_area = (boxes[n, 2] - boxes[n, 0] + 1) * (boxes[n, 3] - boxes[n, 1] + 1)
#                     all_area = float(query_box_area)
#                     overlaps[n, k] = iw * ih / all_area
#     return overlaps

# def _pygenerate(boxes, masks, labels, width, height, chipsize, stride):
#         chips = []
#         boxes = np.array(boxes)
#         masks = np.array(masks)
#         # ensure coverage of image for worst case
#         # corners
#         chips.append([max(width - chipsize, 0), 0, width - 1, min(chipsize, height-1)])
#         chips.append([0, max(height - chipsize, 0), min(chipsize, width-1), height-1])
#         chips.append([max(width - chipsize, 0), max(height - chipsize, 0), width-1, height-1])
		
#         for i in range(0, width - int(chipsize), stride):
#             for j in range(0, height - int(chipsize), stride):
#                 x1 = i
#                 y1 = j
#                 x2 = i + chipsize - 1
#                 y2 = j + chipsize - 1
#                 chips.append([x1, y1, x2, y2])
# 		#width may not be divide by stride
#         for j in range(0, height - int(chipsize), stride):
#             x1 = max(width - chipsize - 1,0)
#             y1 = j
#             x2 = width - 1
#             y2 = j + chipsize - 1
#             chips.append([x1, y1, x2, y2])
# 		#the same as above
#         for i in range(0, width - int(chipsize), stride):
#             x1 = i
#             y1 = max(height - chipsize - 1,0)
#             x2 = i + chipsize - 1
#             y2 = height - 1
#             chips.append([x1, y1, x2, y2])

#         chips = np.array(chips).astype(np.float)
#         overlaps = bbox_overlaps_py(chips, boxes.astype(np.float))
#         fchips = []
#         masks_list = []
#         boxes_list = []
#         labels_list = []
#         for j in range(len(chips)):
#             nvids = np.where(overlaps[j, :] >= 0.9)[0]
#             if(len(nvids) == 0):
#                 continue
#             else:
#                 fchips.append(chips[j])
#                 boxes_list.append(boxes[nvids])
#                 masks_list.append(masks[nvids])
#                 labels_list.append([labels[x] for x in nvids])
#         return fchips, masks_list, boxes_list, labels_list
from split.split import _pygenerate, bbox_overlaps_py
def get_scale_info(fp, scale):
    masks = []
    labels = []
    boxes = []
    for line in fp:
        line = line.strip()
        if not line:
            continue
        items = line.split(' ')
        loc = np.array([float(x) for x in items[:8]])
        loc = loc*scale
        xs = loc[0::2]
        ys = loc[1::2]
        poly = np.array([[[loc[0],loc[1]],[loc[2],loc[3]],[loc[4],loc[5]],[loc[6], loc[7]]]], np.int32)
        area = cv2.contourArea(poly)
        if(area < 50):
            continue
        xmin  = min(xs)
        xmax = max(xs)
        ymin = min(ys)
        ymax = max(ys)
        w = xmax - xmin
        h = ymax - ymin
        cls = items[8]
        if cls not in CLASS:
            continue
        labels.append(cls)
        masks.append(loc)
        boxes.append([xmin,ymin,xmax,ymax])
    return labels, masks, boxes
def construct_imginfo(filename, h, w, ID):
    image = { "license": 1,
              "file_name": filename,
              "coco_url": "xxx",
              "height": h,
              "width": w,
              "date_captured": "2019-06-25",
              "flickr_url": "xxx",
              "id": ID
            }
    return image

def construct_ann(obj_id, ID, category_id, seg, area, bbox):
    ann = {"id" : obj_id, 
                "image_id" : ID, 
                "category_id" : category_id, 
                "segmentation" : seg, 
               "area" :  area, 
               "bbox" : bbox, 
               "iscrowd" : 0,
            }
    return ann

def generate_ann(box, mask, label, xmin, ymin):
    ann = []
    for b, loc, l in zip(box, mask, label):
        x1, y1, x2, y2 = b
        x1 -= xmin
        y1 -= ymin
        x2 -= xmin
        y2 -= ymin
        poly = np.array([[[loc[0],loc[1]],[loc[2],loc[3]],[loc[4],loc[5]],[loc[6], loc[7]]]], np.int32)
        box = [x1, y1, x2, y2]
        area = cv2.contourArea(poly)
        poly = [loc[0]-xmin,loc[1]-ymin,loc[2]-xmin,loc[3]-ymin,loc[4]-xmin,loc[5]-ymin,loc[6]-xmin, loc[7]-ymin]
        obj = [class_to_ind[l], [poly], area, [x1, y1, x2-x1, y2-y1]]
        ann.append(obj)
    return ann

def chip_warpper(mode, chip, imgname, img, box, mask, label, scale, outdir, phase="train", count=0):
    if mode == "multi":
        xmin, ymin, xmax, ymax = chip
        filename = imgname.split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(xmin), str(ymin), str(0))+"part" +str(count) + ".jpg"
        cv2.imwrite("{}/{}".format(outdir, phase)+filename, img)
        h, w,_ = img.shape
        ann = generate_ann(box, mask, label, xmin, ymin)
    else:
        filename = imgname.split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(0), str(0), str(0))+"part" +str(0) + ".jpg"
        h, w,_ = img.shape
        cv2.imwrite("{}/{}".format(outdir, phase)+filename, img)
        ann = generate_ann(box, mask, label, 0, 0)
    return (ann, filename, h, w)
   
from icecream import ic
import time
def scale_generate(datdir, labeldir, phase, outdir, SCALE):
    imgs = os.listdir(datadir)
    info = {
        "description": "rscup",
        "url": "http://cocodataset.org",
        "version": "1.0",
        "year": 2014,
        "contributor": "COCO Consortium",
        "date_created": "2017/09/01"
      }
    license = [{ "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/", "id": 1, "name": "Attribution-NonCommercial-ShareAlike License"} ]
    categories = []
    for cls in CLASS:
        category = { "id" : class_to_ind[cls], "name" : cls, "supercategory" : "object",}
        categories.append(category)
    print(categories)
    annotations = {"info": info, "images": [], "annotations": [], "categories":categories, "license":license}
    obj_id = 0
    ID = 0
    p = Pool(18)
    for img_id in tqdm(range(len(imgs))):
        t0 = time.time()
        img_path = os.path.join(datadir, imgs[img_id])
        label_path = os.path.join(labeldir, imgs[img_id].split(".")[0]+".txt")
        original_img = cv2.imread(img_path)
        scales = copy.deepcopy(SCALE)
        fp = open(label_path).readlines()[2:]
        t1 = time.time()
        ic(t1-t0)
        rets = []
        for scale in scales:
            img, scale_factor = mmcv.imrescale(original_img, scale, return_scale=True)
            H,W,_ = img.shape
            labels, masks, boxes = get_scale_info(fp, scale_factor)
            if(len(labels)<=0):
                continue
            if max(H,W) > 512:
                boxes = np.array(boxes)
                masks = np.array(masks)
                fchips, masks, boxes, labels = _pygenerate(boxes, masks, labels, W, H, 512, 416)
                count = 0
                for chip, mask, box, label in zip(fchips, masks, boxes,labels):
                    count+=1
                    xmin, ymin, xmax, ymax = chip
                    img2 = copy.deepcopy(img[int(ymin):int(ymax), int(xmin):int(xmax),:])
                    rets.append(p.apply_async(chip_warpper, args=("multi", chip, imgs[img_id], img2, box, mask, label, scale, outdir, count)))
            else:
                rets.append(p.apply_async(chip_warpper, args=("single", None, imgs[img_id], img, boxes, masks, labels, scale, outdir, count)))
        t2=time.time()
        ic(t2-t1)
        for ret in rets:
            anns, filename, h, w = ret.get()
            image = construct_imginfo(filename, h, w, ID)
            annotations["images"].append(image)
            for ann in anns:
                category_id, seg, area, bbox = ann
                ann_info = construct_ann(obj_id, ID, category_id, seg, area, bbox)
                annotations["annotations"].append(ann_info)
                obj_id += 1
            ID += 1
        tm = time.time()
        ic
    p.close()
    p.join()
    with open('{}/annos_rscup_'.format(outdir)+phase+'.json', 'w') as json_file:
        json.dump(annotations, json_file, cls=MyEncoder)
    print("totol number {}".format(str(ID)))

In [17]:
CLASS=['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad']
#CLASS={'tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad'}
class_to_ind = dict(zip(CLASS, range(len(CLASS))))
print(class_to_ind)
datadir = "/home/xfr/rssid/data/train/images"
labeldir = "/home/xfr/rssid/data/train/labelTxt"
scale_generate(datadir, labeldir, "train","./data/trash", [1])

{'tennis-court': 0, 'container-crane': 1, 'storage-tank': 2, 'baseball-diamond': 3, 'plane': 4, 'ground-track-field': 5, 'helicopter': 6, 'airport': 7, 'harbor': 8, 'ship': 9, 'large-vehicle': 10, 'swimming-pool': 11, 'soccer-ball-field': 12, 'roundabout': 13, 'basketball-court': 14, 'bridge': 15, 'small-vehicle': 16, 'helipad': 17}
[{'id': 0, 'name': 'tennis-court', 'supercategory': 'object'}, {'id': 1, 'name': 'container-crane', 'supercategory': 'object'}, {'id': 2, 'name': 'storage-tank', 'supercategory': 'object'}, {'id': 3, 'name': 'baseball-diamond', 'supercategory': 'object'}, {'id': 4, 'name': 'plane', 'supercategory': 'object'}, {'id': 5, 'name': 'ground-track-field', 'supercategory': 'object'}, {'id': 6, 'name': 'helicopter', 'supercategory': 'object'}, {'id': 7, 'name': 'airport', 'supercategory': 'object'}, {'id': 8, 'name': 'harbor', 'supercategory': 'object'}, {'id': 9, 'name': 'ship', 'supercategory': 'object'}, {'id': 10, 'name': 'large-vehicle', 'supercategory': 'objec

ic| t1-t0: 0.2205488681793213
ic| t2-t1: 0.1735374927520752
ic| tm-t2: 0.1017465591430664
ic| tm-t0: 0.4958329200744629
ic| t1-t0: 0.17309093475341797
ic| t2-t1: 0.05440878868103027
ic| tm-t2: 0.04949450492858887
ic| tm-t0: 0.2769942283630371
ic| t1-t0: 0.011915922164916992
ic| t2-t1: 0.043721914291381836
ic| tm-t2: 0.046732425689697266
ic| tm-t0: 0.1023702621459961
ic| t1-t0: 0.049788713455200195
ic| t2-t1: 0.047335147857666016
ic| tm-t2: 0.05598258972167969
ic| tm-t0: 0.1531064510345459
ic| t1-t0: 0.08154845237731934
ic| t2-t1: 0.05439186096191406
ic| tm-t2: 0.061335086822509766
ic| tm-t0: 0.19727540016174316
ic| t1-t0: 0.9558184146881104
ic| t2-t1: 0.29990720748901367
ic| tm-t2: 0.14812350273132324
ic| tm-t0: 1.4038491249084473
ic| t1-t0: 0.22947454452514648
ic| t2-t1: 0.04889798164367676
ic| tm-t2: 0.04738450050354004
ic| tm-t0: 0.3257570266723633
ic| t1-t0: 0.03339672088623047
ic| t2-t1: 0.05226492881774902
ic| tm-t2: 0.06541967391967773
ic| tm-t0: 0.15108132362365723
ic| t1-t0: 0

  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/xfr/.c

KeyboardInterrupt: 

  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/xfr/.conda/envs/mmdet/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self

In [73]:
import chips.cchips as chips